# Unit Investment Trust Fund (UITF) Exploration

## Packages

In [1]:
from typing import Union
import re

import pandas as pd
import numpy as np

## Helpers

In [2]:
def remove_char(string: str) -> str:
    pattern = r'[a-zA-Z]'
    return re.sub(pattern, '', string).replace(',', '').strip()

def clean_amount(amount: str) -> str:
    return float(remove_char(amount))


def clean_fee(fee_str):
    if isinstance(fee_str, float):
        return fee_str
        
    match = re.search(r'(\d+(\.\d{1,2})?)%', fee_str)
    if match:
        return float(match.group(1))
    
    match_no_perc = re.search(r'(\d+(\.\d{1,2})?)', fee_str)
    if match_no_perc:
        return float(match_no_perc.group(1))
    
    return None

## Data

In [3]:
df_funds = pd.read_csv('data/funds.csv')
df_performance = pd.read_csv('data/performance.csv')

### Merge dataframes

In [4]:
df = pd.merge(df_funds, df_performance[['fund_name', 'ytd']], left_on='fund_name', right_on='fund_name')

In [5]:
df.columns

Index(['inception_date', 'currency', 'risk_classification',
       'min_initial_participation', 'remarks', 'min_additional_participation',
       'min_maintaining_balance', 'min_holding_period',
       'cutoff_time_for_participationredemption', 'settlement_date',
       'trust_fee_structure', 'early_redemption_fee', 'benchmark', 'fund_name',
       'classification', 'navpu', 'bank', 'url', 'ytd'],
      dtype='object')

In [6]:
df = df.drop(
    [
        'remarks', 
        'cutoff_time_for_participationredemption', 
        'settlement_date', 
        'benchmark', 
        'early_redemption_fee'
    ], 
    axis=1
)

### Cleaning

In [8]:
df['inception_date'] = df['inception_date'].apply(pd.to_datetime)

In [9]:
df[['min_initial_participation', 'min_additional_participation', 'min_maintaining_balance']] = df[['min_initial_participation', 'min_additional_participation', 'min_maintaining_balance']].map(clean_amount)

In [10]:
df['min_holding_period'] = df['min_holding_period'].apply(remove_char)

In [11]:
df['inception_year'] = df['inception_date'].apply(lambda x: x.year)

In [12]:
df['trust_fee_structure'] = df['trust_fee_structure'].apply(clean_fee)

In [14]:
df.head(2)

,inception_date,currency,risk_classification,min_initial_participation,min_additional_participation,min_maintaining_balance,min_holding_period,trust_fee_structure,fund_name,classification,navpu,bank,url,ytd,inception_year
0,2006-12-04,PHP,Aggressive,25000.0,10000.0,25000.0,30,1.25,AB CAPITAL EQUITY FUND,Equity Fund,2.797781,AB Capital Asset Management,https://www.uitf.com.ph/daily_navpu_details.ph...,1.9004,2006
1,2015-12-11,PHP,Balanced,10000.0,10000.0,10000.0,30,1.00,BDO ESG EQUITY FUND,Equity Fund,96.833000,"BDO Unibank, Inc.",https://www.uitf.com.ph/daily_navpu_details.ph...,-4.2677,2015


In [15]:
df.columns

Index(['inception_date', 'currency', 'risk_classification',
       'min_initial_participation', 'min_additional_participation',
       'min_maintaining_balance', 'min_holding_period', 'trust_fee_structure',
       'fund_name', 'classification', 'navpu', 'bank', 'url', 'ytd',
       'inception_year'],
      dtype='object')

In [16]:
df.to_csv('data/data.csv', index=False)